### 173 original month_lag で期間を区切り集計を行う
相対的に時系列のズレはあるが、初期3ヶ月のトランザクションを共通集計し、targetの期間と一致すると考えられるグループで予測モデルを作り、他のグループはその予測モデルで初回のロイヤリティを予測する

In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features, reduce_mem_usage
from preprocessing import get_dummies
import datetime

from tqdm import tqdm
import time
import sys

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

### Data Load

In [71]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
df_train.set_index(key, inplace=True)
df_test.set_index(key, inplace=True)
train_test = pd.concat([df_train, df_test], axis=0)
sys.exit()
df_hist = utils.read_df_pkl('../input/hist_clean_rdm0*')
df_hist = reduce_mem_usage(df_hist)

df_org_lag = utils.read_pkl_gzip('../input/train_test_original_month_lag.gz')
df_org_lag = reduce_mem_usage(df_org_lag)
df_org_lag.head()

df_org_lag.rename(columns={'purchase_date':'yyyymm'}, inplace=True)
print(df_hist.shape)
df_hist = df_hist.merge(df_org_lag.reset_index()[[key, 'yyyymm', 'org_month_lag']], how='inner', on=[key, 'yyyymm'])
print(df_hist.shape)

100%|██████████| 3/3 [00:00<00:00, 128.10it/s]


SystemExit: 

In [73]:
# term
base_month = 0
last_month = 2
col_month = 'org_month_lag'
df = df_hist[df_hist[col_month]<=last_month]
df = df[df[col_month]>=base_month]

auth1 = df[df.authorized_flag==1]
auth0 = df[df.authorized_flag==0]

cat1_0 = False
cat1_0 = True
if cat1_0:
    auth1_cat1 = auth1[auth1.category_1==1]
    auth1_cat0 = auth1[auth1.category_1==0]
    auth0_cat1 = auth0[auth0.category_1==1]
    auth0_cat0 = auth0[auth0.category_1==0]
#     del auth1, auth0
#     gc.collect()

In [74]:
term_type = 'org_lag0_2'
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
df_train.set_index(key, inplace=True)
df_test.set_index(key, inplace=True)
train_test = pd.concat([df_train, df_test], axis=0)

if cat1_0:
    df_list = [auth1_cat1, auth1_cat0, auth0_cat1, auth0_cat0]
    fname_list = [f'{term_type}_auth1_cat1', f'{term_type}_auth1_cat0', f'{term_type}_auth0_cat1', f'{term_type}_auth0_cat0']
else:
    df_list = [auth1, auth0]
    fname_list = [f'{term_type}_auth1', f'{term_type}_auth0']


def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]


# def feat_agg(df, fname):
for df, fname in zip(df_list, fname_list):
#     global train_test
    
    df = pd.get_dummies(df, columns=['category_2', 'category_3'])
    
#     aggs = {}
    if fname.count('cat1'):
        aggs = {
        'category_3_A': ['mean'],
        'category_3_B': ['mean'],
        'category_3_C': ['mean'],
        }
    else:
        aggs = {
        'category_2_1.0': ['mean'],
        'category_2_2.0': ['mean'],
        'category_2_3.0': ['mean'],
        'category_2_4.0': ['mean'],
        'category_2_5.0': ['mean'],
        'category_3_A': ['mean'],
        'category_3_B': ['mean'],
        'category_3_C': ['mean'],
        }
    
    if not(fname.count('cat')):
        aggs['category_1'] = ['sum', 'mean']
            
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['first_active_month'] =  pd.to_datetime(df['first_active_month'])
    df['month_diff'] = (pd.to_datetime('2018-05-01') - df['purchase_date']).dt.days
    
    aggs['month_lag'] = ['mean', 'std']
    aggs['yyyy_week'] = ['nunique']
    aggs['purchase_date'] = ['max','min']
    aggs['month_diff'] = ['mean', 'std']
    
    aggs['purchase_amount'] = ['sum','max', 'min','mean']
    aggs['installments'] = ['sum', 'max', 'min','mean', 'std']
#     aggs['purchase_amount_over'] = ['sum','max', 'min','mean']
    
    aggs['merchant_id'] = ['nunique']
    aggs['merchant_category_id'] = ['nunique']
    aggs['card_id'] = ['size']
    aggs['city_id'] = ['nunique']
    aggs['state_id'] = ['nunique']
    aggs['subsector_id'] = ['nunique']
#     aggs['all_term'] = ['max']
    
    new_columns = get_new_columns(fname, aggs)
    df_agg = df.groupby(key).agg(aggs)
    df_agg.columns = new_columns
    df_agg.reset_index(drop=False,inplace=True)
     
    df_agg[f'{fname}_this_term'] =  (pd.to_datetime(df_agg[f'{fname}_purchase_date_max']) - pd.to_datetime(df_agg[f'{fname}_purchase_date_min'])).dt.days
    df_agg[f'{fname}_freq_per_this_term'] = df_agg[f'{fname}_card_id_size']              / (df_agg[f'{fname}_this_term'] + 1.0)
    df_agg[f'{fname}_amount_per_this_term'] = df_agg[f'{fname}_purchase_amount_sum']     / (df_agg[f'{fname}_this_term'] + 1.0)
    df_agg[f'{fname}_instthisments_per_this_term'] = df_agg[f'{fname}_installments_sum'] / (df_agg[f'{fname}_this_term'] + 1.0)
    
    df_agg[f'{fname}_freq_per_90days'] = df_agg[f'{fname}_card_id_size'] / 90
    df_agg[f'{fname}_amount_per_90days'] = df_agg[f'{fname}_purchase_amount_sum'] / 90
    df_agg[f'{fname}_installments_per_90days'] = df_agg[f'{fname}_installments_sum'] / 90
    
    df_agg[f'{fname}_amount_per_installments_sum'] = df_agg[f'{fname}_purchase_amount_sum'] / (df_agg[f'{fname}_installments_sum'] + 1.0)
    df_agg[f'{fname}_amount_per_installments_mean'] = df_agg[f'{fname}_purchase_amount_mean'] / (df_agg[f'{fname}_installments_mean'] + 1.0)
    
#     df_agg.drop(f'{fname}_all_term_max', axis=1, inplace=True)
    
    # Monthly Agg
#     del aggs['all_term']
    new_columns = get_new_columns(fname + '_monthly_avg', aggs)
    month_agg = df.groupby([key, 'yyyymm']).agg(aggs)
        
    month_agg.columns = new_columns
    month_agg = month_agg.reset_index().drop('yyyymm', axis=1).groupby([key]).mean()
    month_agg.reset_index(drop=False,inplace=True)
    
    train_test = train_test.join(df_agg.set_index(key)).join(month_agg.set_index(key))
    del df_agg, month_agg
    gc.collect()
    
    print(fname)
    print(train_test.shape)
    display(train_test.head())
    
df_feat = train_test

100%|██████████| 3/3 [00:00<00:00, 75.55it/s]


org_lag0_2_auth1_cat1
(325540, 62)


,feature_1,feature_2,feature_3,first_active_month,target,org_lag0_2_auth1_cat1_category_3_A_mean,org_lag0_2_auth1_cat1_category_3_B_mean,org_lag0_2_auth1_cat1_category_3_C_mean,org_lag0_2_auth1_cat1_month_lag_mean,org_lag0_2_auth1_cat1_month_lag_std,org_lag0_2_auth1_cat1_yyyy_week_nunique,org_lag0_2_auth1_cat1_purchase_date_max,org_lag0_2_auth1_cat1_purchase_date_min,org_lag0_2_auth1_cat1_month_diff_mean,org_lag0_2_auth1_cat1_month_diff_std,org_lag0_2_auth1_cat1_purchase_amount_sum,org_lag0_2_auth1_cat1_purchase_amount_max,org_lag0_2_auth1_cat1_purchase_amount_min,org_lag0_2_auth1_cat1_purchase_amount_mean,org_lag0_2_auth1_cat1_installments_sum,org_lag0_2_auth1_cat1_installments_max,org_lag0_2_auth1_cat1_installments_min,org_lag0_2_auth1_cat1_installments_mean,org_lag0_2_auth1_cat1_installments_std,org_lag0_2_auth1_cat1_merchant_id_nunique,org_lag0_2_auth1_cat1_merchant_category_id_nunique,org_lag0_2_auth1_cat1_card_id_size,org_lag0_2_auth1_cat1_city_id_nunique,org_lag0_2_auth1_cat1_state_id_nunique,org_lag0_2_auth1_cat1_subsector_id_nunique,org_lag0_2_auth1_cat1_this_term,org_lag0_2_auth1_cat1_freq_per_this_term,org_lag0_2_auth1_cat1_amount_per_this_term,org_lag0_2_auth1_cat1_instthisments_per_this_term,org_lag0_2_auth1_cat1_freq_per_90days,org_lag0_2_auth1_cat1_amount_per_90days,org_lag0_2_auth1_cat1_installments_per_90days,org_lag0_2_auth1_cat1_amount_per_installments_sum,org_lag0_2_auth1_cat1_amount_per_installments_mean,org_lag0_2_auth1_cat1_monthly_avg_category_3_A_mean,org_lag0_2_auth1_cat1_monthly_avg_category_3_B_mean,org_lag0_2_auth1_cat1_monthly_avg_category_3_C_mean,org_lag0_2_auth1_cat1_monthly_avg_month_lag_mean,org_lag0_2_auth1_cat1_monthly_avg_month_lag_std,org_lag0_2_auth1_cat1_monthly_avg_yyyy_week_nunique,org_lag0_2_auth1_cat1_monthly_avg_month_diff_mean,org_lag0_2_auth1_cat1_monthly_avg_month_diff_std,org_lag0_2_auth1_cat1_monthly_avg_purchase_amount_sum,org_lag0_2_auth1_cat1_monthly_avg_purchase_amount_max,org_lag0_2_auth1_cat1_monthly_avg_purchase_amount_min,org_lag0_2_auth1_cat1_monthly_avg_purchase_amount_mean,org_lag0_2_auth1_cat1_monthly_avg_installments_sum,org_lag0_2_auth1_cat1_monthly_avg_installments_max,org_lag0_2_auth1_cat1_monthly_avg_installments_min,org_lag0_2_auth1_cat1_monthly_avg_installments_mean,org_lag0_2_auth1_cat1_monthly_avg_installments_std,org_lag0_2_auth1_cat1_monthly_avg_merchant_id_nunique,org_lag0_2_auth1_cat1_monthly_avg_merchant_category_id_nunique,org_lag0_2_auth1_cat1_monthly_avg_card_id_size,org_lag0_2_auth1_cat1_monthly_avg_city_id_nunique,org_lag0_2_auth1_cat1_monthly_avg_state_id_nunique,org_lag0_2_auth1_cat1_monthly_avg_subsector_id_nunique
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_92a2005557,5,2,1,2017-06,-0.820283,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C_ID_3d0044924f,4,1,0,2017-01,0.392913,0.0,0.736842,0.263158,-11.368421,0.597265,7.0,2017-03-02 17:04:54,2017-01-10 13:17:29,446.368421,18.108348,3.037109,2.0,0.006020,0.159790,27.0,3.0,1.0,1.420898,0.768555,8.0,8.0,19.0,1.0,1.0,8.0,51.0,0.365385,0.058406,0.519231,0.211111,0.033752,0.300049,0.108459,0.065979,0.0,0.525000,0.475000,-11.0,0.0,2.666667,440.966667,5.610303,1.011719,0.811523,0.064087,0.172485,9.0,2.666016,1.666992,1.875000,0.601562,3.333333,3.333333,6.333333,1.0,1.0,3.333333
C_ID_d639edf6cd,2,2,0,2016-08,0.688056,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C_ID_186d6a6901,4,3,0,2017-09,0.142495,0.0,0.200000,0.800000,-3.400000,0.547723,4.0,2017-11-24 11:32:02,2017-10-10 17:33:48,175.200000,22.015903,3.222656,2.0,0.176270,0.644531,12.0,3.0,1.0,2.400391,0.894531,5.0,5.0,5.0,1.0,1.0,4.0,44.0,0.111111,0.071615,0.26

org_lag0_2_auth1_cat0
(325540, 129)


,feature_1,feature_2,feature_3,first_active_month,target,org_lag0_2_auth1_cat1_category_3_A_mean,org_lag0_2_auth1_cat1_category_3_B_mean,org_lag0_2_auth1_cat1_category_3_C_mean,org_lag0_2_auth1_cat1_month_lag_mean,org_lag0_2_auth1_cat1_month_lag_std,org_lag0_2_auth1_cat1_yyyy_week_nunique,org_lag0_2_auth1_cat1_purchase_date_max,org_lag0_2_auth1_cat1_purchase_date_min,org_lag0_2_auth1_cat1_month_diff_mean,org_lag0_2_auth1_cat1_month_diff_std,org_lag0_2_auth1_cat1_purchase_amount_sum,org_lag0_2_auth1_cat1_purchase_amount_max,org_lag0_2_auth1_cat1_purchase_amount_min,org_lag0_2_auth1_cat1_purchase_amount_mean,org_lag0_2_auth1_cat1_installments_sum,org_lag0_2_auth1_cat1_installments_max,org_lag0_2_auth1_cat1_installments_min,org_lag0_2_auth1_cat1_installments_mean,org_lag0_2_auth1_cat1_installments_std,org_lag0_2_auth1_cat1_merchant_id_nunique,org_lag0_2_auth1_cat1_merchant_category_id_nunique,org_lag0_2_auth1_cat1_card_id_size,org_lag0_2_auth1_cat1_city_id_nunique,org_lag0_2_auth1_cat1_state_id_nunique,org_lag0_2_auth1_cat1_subsector_id_nunique,org_lag0_2_auth1_cat1_this_term,org_lag0_2_auth1_cat1_freq_per_this_term,org_lag0_2_auth1_cat1_amount_per_this_term,org_lag0_2_auth1_cat1_instthisments_per_this_term,org_lag0_2_auth1_cat1_freq_per_90days,org_lag0_2_auth1_cat1_amount_per_90days,org_lag0_2_auth1_cat1_installments_per_90days,org_lag0_2_auth1_cat1_amount_per_installments_sum,org_lag0_2_auth1_cat1_amount_per_installments_mean,org_lag0_2_auth1_cat1_monthly_avg_category_3_A_mean,org_lag0_2_auth1_cat1_monthly_avg_category_3_B_mean,org_lag0_2_auth1_cat1_monthly_avg_category_3_C_mean,org_lag0_2_auth1_cat1_monthly_avg_month_lag_mean,org_lag0_2_auth1_cat1_monthly_avg_month_lag_std,org_lag0_2_auth1_cat1_monthly_avg_yyyy_week_nunique,org_lag0_2_auth1_cat1_monthly_avg_month_diff_mean,org_lag0_2_auth1_cat1_monthly_avg_month_diff_std,org_lag0_2_auth1_cat1_monthly_avg_purchase_amount_sum,org_lag0_2_auth1_cat1_monthly_avg_purchase_amount_max,org_lag0_2_auth1_cat1_monthly_avg_purchase_amount_min,org_lag0_2_auth1_cat1_monthly_avg_purchase_amount_mean,org_lag0_2_auth1_cat1_monthly_avg_installments_sum,org_lag0_2_auth1_cat1_monthly_avg_installments_max,org_lag0_2_auth1_cat1_monthly_avg_installments_min,org_lag0_2_auth1_cat1_monthly_avg_installments_mean,org_lag0_2_auth1_cat1_monthly_avg_installments_std,org_lag0_2_auth1_cat1_monthly_avg_merchant_id_nunique,org_lag0_2_auth1_cat1_monthly_avg_merchant_category_id_nunique,org_lag0_2_auth1_cat1_monthly_avg_card_id_size,org_lag0_2_auth1_cat1_monthly_avg_city_id_nunique,org_lag0_2_auth1_cat1_monthly_avg_state_id_nunique,org_lag0_2_auth1_cat1_monthly_avg_subsector_id_nunique,org_lag0_2_auth1_cat0_category_2_1.0_mean,org_lag0_2_auth1_cat0_category_2_2.0_mean,org_lag0_2_auth1_cat0_category_2_3.0_mean,org_lag0_2_auth1_cat0_category_2_4.0_mean,org_lag0_2_auth1_cat0_category_2_5.0_mean,org_lag0_2_auth1_cat0_category_3_A_mean,org_lag0_2_auth1_cat0_category_3_B_mean,org_lag0_2_auth1_cat0_category_3_C_mean,org_lag0_2_auth1_cat0_month_lag_mean,org_lag0_2_auth1_cat0_month_lag_std,org_lag0_2_auth1_cat0_yyyy_week_nunique,org_lag0_2_auth1_cat0_purchase_date_max,org_lag0_2_auth1_cat0_purchase_date_min,org_lag0_2_auth1_cat0_month_diff_mean,org_lag0_2_auth1_cat0_month_diff_std,org_lag0_2_auth1_cat0_purchase_amount_sum,org_lag0_2_auth1_cat0_purchase_amount_max,org_lag0_2_auth1_cat0_purchase_amount_min,org_lag0_2_auth1_cat0_purchase_amount_mean,org_lag0_2_auth1_cat0_installments_sum,org_lag0_2_auth1_cat0_installments_max,org_lag0_2_auth1_cat0_installments_min,org_lag0_2_auth1_cat0_installments_mean,org_lag0_2_auth1_cat0_installments_std,org_lag0_2_auth1_cat0_merchant_id_nunique,org_lag0_2_auth1_cat0_merchant_category_id_nunique,org_lag0_2_auth1_cat0_card_id_size,org_lag0_2_auth1_cat0_city_id_nunique,org_lag0_2_auth1_cat0_state_id_nunique,org_lag0_2_auth1_cat0_subsector_id_nunique,org_lag0_2_auth1_cat0_this_term,org_lag0_2_auth1_cat0_freq_per_this_term,org_lag0_2_auth1_cat0_amount_per_this_term,org_lag0_2_auth1_cat0_instt

org_lag0_2_auth0_cat1
(325540, 186)


,feature_1,feature_2,feature_3,first_active_month,target,org_lag0_2_auth1_cat1_category_3_A_mean,org_lag0_2_auth1_cat1_category_3_B_mean,org_lag0_2_auth1_cat1_category_3_C_mean,org_lag0_2_auth1_cat1_month_lag_mean,org_lag0_2_auth1_cat1_month_lag_std,org_lag0_2_auth1_cat1_yyyy_week_nunique,org_lag0_2_auth1_cat1_purchase_date_max,org_lag0_2_auth1_cat1_purchase_date_min,org_lag0_2_auth1_cat1_month_diff_mean,org_lag0_2_auth1_cat1_month_diff_std,org_lag0_2_auth1_cat1_purchase_amount_sum,org_lag0_2_auth1_cat1_purchase_amount_max,org_lag0_2_auth1_cat1_purchase_amount_min,org_lag0_2_auth1_cat1_purchase_amount_mean,org_lag0_2_auth1_cat1_installments_sum,org_lag0_2_auth1_cat1_installments_max,org_lag0_2_auth1_cat1_installments_min,org_lag0_2_auth1_cat1_installments_mean,org_lag0_2_auth1_cat1_installments_std,org_lag0_2_auth1_cat1_merchant_id_nunique,org_lag0_2_auth1_cat1_merchant_category_id_nunique,org_lag0_2_auth1_cat1_card_id_size,org_lag0_2_auth1_cat1_city_id_nunique,org_lag0_2_auth1_cat1_state_id_nunique,org_lag0_2_auth1_cat1_subsector_id_nunique,org_lag0_2_auth1_cat1_this_term,org_lag0_2_auth1_cat1_freq_per_this_term,org_lag0_2_auth1_cat1_amount_per_this_term,org_lag0_2_auth1_cat1_instthisments_per_this_term,org_lag0_2_auth1_cat1_freq_per_90days,org_lag0_2_auth1_cat1_amount_per_90days,org_lag0_2_auth1_cat1_installments_per_90days,org_lag0_2_auth1_cat1_amount_per_installments_sum,org_lag0_2_auth1_cat1_amount_per_installments_mean,org_lag0_2_auth1_cat1_monthly_avg_category_3_A_mean,org_lag0_2_auth1_cat1_monthly_avg_category_3_B_mean,org_lag0_2_auth1_cat1_monthly_avg_category_3_C_mean,org_lag0_2_auth1_cat1_monthly_avg_month_lag_mean,org_lag0_2_auth1_cat1_monthly_avg_month_lag_std,org_lag0_2_auth1_cat1_monthly_avg_yyyy_week_nunique,org_lag0_2_auth1_cat1_monthly_avg_month_diff_mean,org_lag0_2_auth1_cat1_monthly_avg_month_diff_std,org_lag0_2_auth1_cat1_monthly_avg_purchase_amount_sum,org_lag0_2_auth1_cat1_monthly_avg_purchase_amount_max,org_lag0_2_auth1_cat1_monthly_avg_purchase_amount_min,org_lag0_2_auth1_cat1_monthly_avg_purchase_amount_mean,org_lag0_2_auth1_cat1_monthly_avg_installments_sum,org_lag0_2_auth1_cat1_monthly_avg_installments_max,org_lag0_2_auth1_cat1_monthly_avg_installments_min,org_lag0_2_auth1_cat1_monthly_avg_installments_mean,org_lag0_2_auth1_cat1_monthly_avg_installments_std,org_lag0_2_auth1_cat1_monthly_avg_merchant_id_nunique,org_lag0_2_auth1_cat1_monthly_avg_merchant_category_id_nunique,org_lag0_2_auth1_cat1_monthly_avg_card_id_size,org_lag0_2_auth1_cat1_monthly_avg_city_id_nunique,org_lag0_2_auth1_cat1_monthly_avg_state_id_nunique,org_lag0_2_auth1_cat1_monthly_avg_subsector_id_nunique,org_lag0_2_auth1_cat0_category_2_1.0_mean,org_lag0_2_auth1_cat0_category_2_2.0_mean,org_lag0_2_auth1_cat0_category_2_3.0_mean,org_lag0_2_auth1_cat0_category_2_4.0_mean,org_lag0_2_auth1_cat0_category_2_5.0_mean,org_lag0_2_auth1_cat0_category_3_A_mean,org_lag0_2_auth1_cat0_category_3_B_mean,org_lag0_2_auth1_cat0_category_3_C_mean,org_lag0_2_auth1_cat0_month_lag_mean,org_lag0_2_auth1_cat0_month_lag_std,org_lag0_2_auth1_cat0_yyyy_week_nunique,org_lag0_2_auth1_cat0_purchase_date_max,org_lag0_2_auth1_cat0_purchase_date_min,org_lag0_2_auth1_cat0_month_diff_mean,org_lag0_2_auth1_cat0_month_diff_std,org_lag0_2_auth1_cat0_purchase_amount_sum,org_lag0_2_auth1_cat0_purchase_amount_max,org_lag0_2_auth1_cat0_purchase_amount_min,org_lag0_2_auth1_cat0_purchase_amount_mean,org_lag0_2_auth1_cat0_installments_sum,org_lag0_2_auth1_cat0_installments_max,org_lag0_2_auth1_cat0_installments_min,org_lag0_2_auth1_cat0_installments_mean,org_lag0_2_auth1_cat0_installments_std,org_lag0_2_auth1_cat0_merchant_id_nunique,org_lag0_2_auth1_cat0_merchant_category_id_nunique,org_lag0_2_auth1_cat0_card_id_size,org_lag0_2_auth1_cat0_city_id_nunique,org_lag0_2_auth1_cat0_state_id_nunique,org_lag0_2_auth1_cat0_subsector_id_nunique,org_lag0_2_auth1_cat0_this_term,org_lag0_2_auth1_cat0_freq_per_this_term,org_lag0_2_auth1_cat0_amount_per_this_term,org_lag0_2_auth1_cat0_instt

org_lag0_2_auth0_cat0
(325540, 253)


,feature_1,feature_2,feature_3,first_active_month,target,org_lag0_2_auth1_cat1_category_3_A_mean,org_lag0_2_auth1_cat1_category_3_B_mean,org_lag0_2_auth1_cat1_category_3_C_mean,org_lag0_2_auth1_cat1_month_lag_mean,org_lag0_2_auth1_cat1_month_lag_std,org_lag0_2_auth1_cat1_yyyy_week_nunique,org_lag0_2_auth1_cat1_purchase_date_max,org_lag0_2_auth1_cat1_purchase_date_min,org_lag0_2_auth1_cat1_month_diff_mean,org_lag0_2_auth1_cat1_month_diff_std,org_lag0_2_auth1_cat1_purchase_amount_sum,org_lag0_2_auth1_cat1_purchase_amount_max,org_lag0_2_auth1_cat1_purchase_amount_min,org_lag0_2_auth1_cat1_purchase_amount_mean,org_lag0_2_auth1_cat1_installments_sum,org_lag0_2_auth1_cat1_installments_max,org_lag0_2_auth1_cat1_installments_min,org_lag0_2_auth1_cat1_installments_mean,org_lag0_2_auth1_cat1_installments_std,org_lag0_2_auth1_cat1_merchant_id_nunique,org_lag0_2_auth1_cat1_merchant_category_id_nunique,org_lag0_2_auth1_cat1_card_id_size,org_lag0_2_auth1_cat1_city_id_nunique,org_lag0_2_auth1_cat1_state_id_nunique,org_lag0_2_auth1_cat1_subsector_id_nunique,org_lag0_2_auth1_cat1_this_term,org_lag0_2_auth1_cat1_freq_per_this_term,org_lag0_2_auth1_cat1_amount_per_this_term,org_lag0_2_auth1_cat1_instthisments_per_this_term,org_lag0_2_auth1_cat1_freq_per_90days,org_lag0_2_auth1_cat1_amount_per_90days,org_lag0_2_auth1_cat1_installments_per_90days,org_lag0_2_auth1_cat1_amount_per_installments_sum,org_lag0_2_auth1_cat1_amount_per_installments_mean,org_lag0_2_auth1_cat1_monthly_avg_category_3_A_mean,org_lag0_2_auth1_cat1_monthly_avg_category_3_B_mean,org_lag0_2_auth1_cat1_monthly_avg_category_3_C_mean,org_lag0_2_auth1_cat1_monthly_avg_month_lag_mean,org_lag0_2_auth1_cat1_monthly_avg_month_lag_std,org_lag0_2_auth1_cat1_monthly_avg_yyyy_week_nunique,org_lag0_2_auth1_cat1_monthly_avg_month_diff_mean,org_lag0_2_auth1_cat1_monthly_avg_month_diff_std,org_lag0_2_auth1_cat1_monthly_avg_purchase_amount_sum,org_lag0_2_auth1_cat1_monthly_avg_purchase_amount_max,org_lag0_2_auth1_cat1_monthly_avg_purchase_amount_min,org_lag0_2_auth1_cat1_monthly_avg_purchase_amount_mean,org_lag0_2_auth1_cat1_monthly_avg_installments_sum,org_lag0_2_auth1_cat1_monthly_avg_installments_max,org_lag0_2_auth1_cat1_monthly_avg_installments_min,org_lag0_2_auth1_cat1_monthly_avg_installments_mean,org_lag0_2_auth1_cat1_monthly_avg_installments_std,org_lag0_2_auth1_cat1_monthly_avg_merchant_id_nunique,org_lag0_2_auth1_cat1_monthly_avg_merchant_category_id_nunique,org_lag0_2_auth1_cat1_monthly_avg_card_id_size,org_lag0_2_auth1_cat1_monthly_avg_city_id_nunique,org_lag0_2_auth1_cat1_monthly_avg_state_id_nunique,org_lag0_2_auth1_cat1_monthly_avg_subsector_id_nunique,org_lag0_2_auth1_cat0_category_2_1.0_mean,org_lag0_2_auth1_cat0_category_2_2.0_mean,org_lag0_2_auth1_cat0_category_2_3.0_mean,org_lag0_2_auth1_cat0_category_2_4.0_mean,org_lag0_2_auth1_cat0_category_2_5.0_mean,org_lag0_2_auth1_cat0_category_3_A_mean,org_lag0_2_auth1_cat0_category_3_B_mean,org_lag0_2_auth1_cat0_category_3_C_mean,org_lag0_2_auth1_cat0_month_lag_mean,org_lag0_2_auth1_cat0_month_lag_std,org_lag0_2_auth1_cat0_yyyy_week_nunique,org_lag0_2_auth1_cat0_purchase_date_max,org_lag0_2_auth1_cat0_purchase_date_min,org_lag0_2_auth1_cat0_month_diff_mean,org_lag0_2_auth1_cat0_month_diff_std,org_lag0_2_auth1_cat0_purchase_amount_sum,org_lag0_2_auth1_cat0_purchase_amount_max,org_lag0_2_auth1_cat0_purchase_amount_min,org_lag0_2_auth1_cat0_purchase_amount_mean,org_lag0_2_auth1_cat0_installments_sum,org_lag0_2_auth1_cat0_installments_max,org_lag0_2_auth1_cat0_installments_min,org_lag0_2_auth1_cat0_installments_mean,org_lag0_2_auth1_cat0_installments_std,org_lag0_2_auth1_cat0_merchant_id_nunique,org_lag0_2_auth1_cat0_merchant_category_id_nunique,org_lag0_2_auth1_cat0_card_id_size,org_lag0_2_auth1_cat0_city_id_nunique,org_lag0_2_auth1_cat0_state_id_nunique,org_lag0_2_auth1_cat0_subsector_id_nunique,org_lag0_2_auth1_cat0_this_term,org_lag0_2_auth1_cat0_freq_per_this_term,org_lag0_2_auth1_cat0_amount_per_this_term,org_lag0_2_auth1_cat0_instt

In [75]:
# impute

def impute_agg(df):
    for col in df.columns:
        if col.count('isnull'):
            df[col].fillna(-1, inplace=True)
        if col.count('null_cnt'):
            df[col].fillna(-1, inplace=True)
        if col.count('month_lag'):
            df[col].fillna(df[col].min()-15, inplace=True)
        if col.count('month_diff'):
            df[col].fillna(df[col].max()+100, inplace=True)
        if col.count('nunique'):
            df[col].fillna(-1, inplace=True)
        if col.count('amount'):
            df[col].fillna(-1, inplace=True)
        if col.count('installments'):
            df[col].fillna(-2, inplace=True)
        if col.count('unix_date'):
            df[col].fillna(df[col].min()-100000, inplace=True)
        if col.count('size'):
            df[col].fillna(-1, inplace=True)
        if col.count('term'):
            df[col].fillna(-1, inplace=True)
        if col.count('per'):
            df[col].fillna(-1, inplace=True)
        if col.count('dummie'):
            df[col].fillna(-1, inplace=True)
            
    return df


df_feat = impute_agg(df_feat)

In [76]:
term_type = 'org_lag0_2'

if cat1_0:
    fname_list = [f'{term_type}_auth1_cat1', f'{term_type}_auth1_cat0', f'{term_type}_auth0_cat1', f'{term_type}_auth0_cat0']
#     fname_list = [f'{term_type}_auth1_cat1', f'{term_type}_auth1_cat0', f'{term_type}_auth0_cat1', f'{term_type}_auth0_cat0', f'{term_type}_new_cat1', f'{term_type}_new_cat0']
else:
#     fname_list = [f'{term_type}_auth1', f'{term_type}_auth0', f'{term_type}_new']
    fname_list = [f'{term_type}_auth1', f'{term_type}_auth0']

    
for fname in fname_list:

    df_feat['first_active_month'] =  pd.to_datetime(df_feat['first_active_month'])
    for col in df_feat.columns:
        if col.count('purchase_date'):
            df_feat[col] = pd.to_datetime( df_feat[col])
    
    # 最終のcat0, auth1までの期間
    df_feat[f'{fname}_term_from_first_month'] = (df_feat[f'{fname}_purchase_date_max'] - df_feat[f'first_active_month']).dt.days
    
    # date_max~date_minの期間
#     df_feat[f'{fname}_this_term'] = (df_feat[f'{fname}_purchase_date_max'] - df_feat[f'{fname}_purchase_date_min']).dt.days
    
    # 20180501までの期間を各データセットパターンで
    df_feat[f'{fname}_term_from_now'] = (pd.to_datetime('2018-05-01') - df_feat[f'{fname}_purchase_date_max']).dt.days
    
    
# auth1, auth0の間の期間
if cat1_0:
    df_feat[f'{term_type}_auth1_0_cat0_term'] = df_feat[f'{term_type}_auth1_cat0_term_from_now'] - df_feat[f'{term_type}_auth0_cat0_term_from_now']
    df_feat[f'{term_type}_auth1_0_cat1_term'] = df_feat[f'{term_type}_auth1_cat1_term_from_now'] - df_feat[f'{term_type}_auth0_cat1_term_from_now']
else:
    df_feat[f'{term_type}_auth1_0_term'] = df_feat[f'{term_type}_auth1_term_from_now'] - df_feat[f'{term_type}_auth0_term_from_now']
    df_feat[f'{term_type}_auth1_0_term'] = df_feat[f'{term_type}_auth1_term_from_now'] - df_feat[f'{term_type}_auth0_term_from_now']
    df_feat[f'{term_type}_auth1_0_purchase_date_max_diff'] = (df_feat[f'{term_type}_auth1_purchase_date_max'] - df_feat[f'{term_type}_auth0_purchase_date_max']).dt.days
    df_feat[f'{term_type}_auth1_0_purchase_date_min_diff'] = (df_feat[f'{term_type}_auth1_purchase_date_min'] - df_feat[f'{term_type}_auth0_purchase_date_min']).dt.days
    
#     df_feat[f'{term_type}_new_auth1_term'] = df_feat[f'{term_type}_new_term_from_now'] - df_feat[f'{term_type}_auth1_term_from_now']
#     df_feat[f'{term_type}_new_min_auth1_max_date_diff'] = (df_feat[f'all_new_purchase_date_min'] - df_feat[f'all_auth1_purchase_date_max']).dt.days
#     df_feat[f'{term_type}_new_min_auth1_min_date_diff'] = (df_feat[f'all_new_purchase_date_min'] - df_feat[f'all_auth1_purchase_date_min']).dt.days
    
# common feature
df_feat[f'first_active_month'].fillna('2017-03', inplace=True)
df_feat[f'first_active_month'] =  pd.to_datetime(df_feat['first_active_month'])

df_feat[f'{term_type}_elapsed_time'] = (pd.to_datetime('2018-05-01') - df_feat['first_active_month']).dt.days
date_min_cols = [col for col in df_feat.columns if col.count('purchase_date_min') and not(col.count('diff'))]
for col in date_min_cols:
    fname = ''
    if col.count('auth1'):
        fname += 'auth1_'
    elif col.count('auth0'):
        fname += 'auth0_'
    if col.count('new'):
        fname += 'new_'
    if col.count('cat1'):
        fname += 'cat1_'
    elif col.count('cat0'):
        fname += 'cat0_'
    df_feat[f'{term_type}_{fname}first_buy'] = (df_feat[col] - df_feat[f'first_active_month']).dt.days
df_feat.head()

,feature_1,feature_2,feature_3,first_active_month,target,org_lag0_2_auth1_cat1_category_3_A_mean,org_lag0_2_auth1_cat1_category_3_B_mean,org_lag0_2_auth1_cat1_category_3_C_mean,org_lag0_2_auth1_cat1_month_lag_mean,org_lag0_2_auth1_cat1_month_lag_std,org_lag0_2_auth1_cat1_yyyy_week_nunique,org_lag0_2_auth1_cat1_purchase_date_max,org_lag0_2_auth1_cat1_purchase_date_min,org_lag0_2_auth1_cat1_month_diff_mean,org_lag0_2_auth1_cat1_month_diff_std,org_lag0_2_auth1_cat1_purchase_amount_sum,org_lag0_2_auth1_cat1_purchase_amount_max,org_lag0_2_auth1_cat1_purchase_amount_min,org_lag0_2_auth1_cat1_purchase_amount_mean,org_lag0_2_auth1_cat1_installments_sum,org_lag0_2_auth1_cat1_installments_max,org_lag0_2_auth1_cat1_installments_min,org_lag0_2_auth1_cat1_installments_mean,org_lag0_2_auth1_cat1_installments_std,org_lag0_2_auth1_cat1_merchant_id_nunique,org_lag0_2_auth1_cat1_merchant_category_id_nunique,org_lag0_2_auth1_cat1_card_id_size,org_lag0_2_auth1_cat1_city_id_nunique,org_lag0_2_auth1_cat1_state_id_nunique,org_lag0_2_auth1_cat1_subsector_id_nunique,org_lag0_2_auth1_cat1_this_term,org_lag0_2_auth1_cat1_freq_per_this_term,org_lag0_2_auth1_cat1_amount_per_this_term,org_lag0_2_auth1_cat1_instthisments_per_this_term,org_lag0_2_auth1_cat1_freq_per_90days,org_lag0_2_auth1_cat1_amount_per_90days,org_lag0_2_auth1_cat1_installments_per_90days,org_lag0_2_auth1_cat1_amount_per_installments_sum,org_lag0_2_auth1_cat1_amount_per_installments_mean,org_lag0_2_auth1_cat1_monthly_avg_category_3_A_mean,org_lag0_2_auth1_cat1_monthly_avg_category_3_B_mean,org_lag0_2_auth1_cat1_monthly_avg_category_3_C_mean,org_lag0_2_auth1_cat1_monthly_avg_month_lag_mean,org_lag0_2_auth1_cat1_monthly_avg_month_lag_std,org_lag0_2_auth1_cat1_monthly_avg_yyyy_week_nunique,org_lag0_2_auth1_cat1_monthly_avg_month_diff_mean,org_lag0_2_auth1_cat1_monthly_avg_month_diff_std,org_lag0_2_auth1_cat1_monthly_avg_purchase_amount_sum,org_lag0_2_auth1_cat1_monthly_avg_purchase_amount_max,org_lag0_2_auth1_cat1_monthly_avg_purchase_amount_min,org_lag0_2_auth1_cat1_monthly_avg_purchase_amount_mean,org_lag0_2_auth1_cat1_monthly_avg_installments_sum,org_lag0_2_auth1_cat1_monthly_avg_installments_max,org_lag0_2_auth1_cat1_monthly_avg_installments_min,org_lag0_2_auth1_cat1_monthly_avg_installments_mean,org_lag0_2_auth1_cat1_monthly_avg_installments_std,org_lag0_2_auth1_cat1_monthly_avg_merchant_id_nunique,org_lag0_2_auth1_cat1_monthly_avg_merchant_category_id_nunique,org_lag0_2_auth1_cat1_monthly_avg_card_id_size,org_lag0_2_auth1_cat1_monthly_avg_city_id_nunique,org_lag0_2_auth1_cat1_monthly_avg_state_id_nunique,org_lag0_2_auth1_cat1_monthly_avg_subsector_id_nunique,org_lag0_2_auth1_cat0_category_2_1.0_mean,org_lag0_2_auth1_cat0_category_2_2.0_mean,org_lag0_2_auth1_cat0_category_2_3.0_mean,org_lag0_2_auth1_cat0_category_2_4.0_mean,org_lag0_2_auth1_cat0_category_2_5.0_mean,org_lag0_2_auth1_cat0_category_3_A_mean,org_lag0_2_auth1_cat0_category_3_B_mean,org_lag0_2_auth1_cat0_category_3_C_mean,org_lag0_2_auth1_cat0_month_lag_mean,org_lag0_2_auth1_cat0_month_lag_std,org_lag0_2_auth1_cat0_yyyy_week_nunique,org_lag0_2_auth1_cat0_purchase_date_max,org_lag0_2_auth1_cat0_purchase_date_min,org_lag0_2_auth1_cat0_month_diff_mean,org_lag0_2_auth1_cat0_month_diff_std,org_lag0_2_auth1_cat0_purchase_amount_sum,org_lag0_2_auth1_cat0_purchase_amount_max,org_lag0_2_auth1_cat0_purchase_amount_min,org_lag0_2_auth1_cat0_purchase_amount_mean,org_lag0_2_auth1_cat0_installments_sum,org_lag0_2_auth1_cat0_installments_max,org_lag0_2_auth1_cat0_installments_min,org_lag0_2_auth1_cat0_installments_mean,org_lag0_2_auth1_cat0_installments_std,org_lag0_2_auth1_cat0_merchant_id_nunique,org_lag0_2_auth1_cat0_merchant_category_id_nunique,org_lag0_2_auth1_cat0_card_id_size,org_lag0_2_auth1_cat0_city_id_nunique,org_lag0_2_auth1_cat0_state_id_nunique,org_lag0_2_auth1_cat0_subsector_id_nunique,org_lag0_2_auth1_cat0_this_term,org_lag0_2_auth1_cat0_freq_per_this_term,org_lag0_2_auth1_cat0_amount_per_this_term,org_lag0_2_auth1_cat0_instt

In [77]:
# Null Impute
for col in df_feat.columns:
    if col in ignore_list:continue
    if str(type(df_feat[col].dtypes)).count('time'):
        df_feat[col].fillna(df_feat[col].min(), inplace=True)
df_feat.head()

,feature_1,feature_2,feature_3,first_active_month,target,org_lag0_2_auth1_cat1_category_3_A_mean,org_lag0_2_auth1_cat1_category_3_B_mean,org_lag0_2_auth1_cat1_category_3_C_mean,org_lag0_2_auth1_cat1_month_lag_mean,org_lag0_2_auth1_cat1_month_lag_std,org_lag0_2_auth1_cat1_yyyy_week_nunique,org_lag0_2_auth1_cat1_purchase_date_max,org_lag0_2_auth1_cat1_purchase_date_min,org_lag0_2_auth1_cat1_month_diff_mean,org_lag0_2_auth1_cat1_month_diff_std,org_lag0_2_auth1_cat1_purchase_amount_sum,org_lag0_2_auth1_cat1_purchase_amount_max,org_lag0_2_auth1_cat1_purchase_amount_min,org_lag0_2_auth1_cat1_purchase_amount_mean,org_lag0_2_auth1_cat1_installments_sum,org_lag0_2_auth1_cat1_installments_max,org_lag0_2_auth1_cat1_installments_min,org_lag0_2_auth1_cat1_installments_mean,org_lag0_2_auth1_cat1_installments_std,org_lag0_2_auth1_cat1_merchant_id_nunique,org_lag0_2_auth1_cat1_merchant_category_id_nunique,org_lag0_2_auth1_cat1_card_id_size,org_lag0_2_auth1_cat1_city_id_nunique,org_lag0_2_auth1_cat1_state_id_nunique,org_lag0_2_auth1_cat1_subsector_id_nunique,org_lag0_2_auth1_cat1_this_term,org_lag0_2_auth1_cat1_freq_per_this_term,org_lag0_2_auth1_cat1_amount_per_this_term,org_lag0_2_auth1_cat1_instthisments_per_this_term,org_lag0_2_auth1_cat1_freq_per_90days,org_lag0_2_auth1_cat1_amount_per_90days,org_lag0_2_auth1_cat1_installments_per_90days,org_lag0_2_auth1_cat1_amount_per_installments_sum,org_lag0_2_auth1_cat1_amount_per_installments_mean,org_lag0_2_auth1_cat1_monthly_avg_category_3_A_mean,org_lag0_2_auth1_cat1_monthly_avg_category_3_B_mean,org_lag0_2_auth1_cat1_monthly_avg_category_3_C_mean,org_lag0_2_auth1_cat1_monthly_avg_month_lag_mean,org_lag0_2_auth1_cat1_monthly_avg_month_lag_std,org_lag0_2_auth1_cat1_monthly_avg_yyyy_week_nunique,org_lag0_2_auth1_cat1_monthly_avg_month_diff_mean,org_lag0_2_auth1_cat1_monthly_avg_month_diff_std,org_lag0_2_auth1_cat1_monthly_avg_purchase_amount_sum,org_lag0_2_auth1_cat1_monthly_avg_purchase_amount_max,org_lag0_2_auth1_cat1_monthly_avg_purchase_amount_min,org_lag0_2_auth1_cat1_monthly_avg_purchase_amount_mean,org_lag0_2_auth1_cat1_monthly_avg_installments_sum,org_lag0_2_auth1_cat1_monthly_avg_installments_max,org_lag0_2_auth1_cat1_monthly_avg_installments_min,org_lag0_2_auth1_cat1_monthly_avg_installments_mean,org_lag0_2_auth1_cat1_monthly_avg_installments_std,org_lag0_2_auth1_cat1_monthly_avg_merchant_id_nunique,org_lag0_2_auth1_cat1_monthly_avg_merchant_category_id_nunique,org_lag0_2_auth1_cat1_monthly_avg_card_id_size,org_lag0_2_auth1_cat1_monthly_avg_city_id_nunique,org_lag0_2_auth1_cat1_monthly_avg_state_id_nunique,org_lag0_2_auth1_cat1_monthly_avg_subsector_id_nunique,org_lag0_2_auth1_cat0_category_2_1.0_mean,org_lag0_2_auth1_cat0_category_2_2.0_mean,org_lag0_2_auth1_cat0_category_2_3.0_mean,org_lag0_2_auth1_cat0_category_2_4.0_mean,org_lag0_2_auth1_cat0_category_2_5.0_mean,org_lag0_2_auth1_cat0_category_3_A_mean,org_lag0_2_auth1_cat0_category_3_B_mean,org_lag0_2_auth1_cat0_category_3_C_mean,org_lag0_2_auth1_cat0_month_lag_mean,org_lag0_2_auth1_cat0_month_lag_std,org_lag0_2_auth1_cat0_yyyy_week_nunique,org_lag0_2_auth1_cat0_purchase_date_max,org_lag0_2_auth1_cat0_purchase_date_min,org_lag0_2_auth1_cat0_month_diff_mean,org_lag0_2_auth1_cat0_month_diff_std,org_lag0_2_auth1_cat0_purchase_amount_sum,org_lag0_2_auth1_cat0_purchase_amount_max,org_lag0_2_auth1_cat0_purchase_amount_min,org_lag0_2_auth1_cat0_purchase_amount_mean,org_lag0_2_auth1_cat0_installments_sum,org_lag0_2_auth1_cat0_installments_max,org_lag0_2_auth1_cat0_installments_min,org_lag0_2_auth1_cat0_installments_mean,org_lag0_2_auth1_cat0_installments_std,org_lag0_2_auth1_cat0_merchant_id_nunique,org_lag0_2_auth1_cat0_merchant_category_id_nunique,org_lag0_2_auth1_cat0_card_id_size,org_lag0_2_auth1_cat0_city_id_nunique,org_lag0_2_auth1_cat0_state_id_nunique,org_lag0_2_auth1_cat0_subsector_id_nunique,org_lag0_2_auth1_cat0_this_term,org_lag0_2_auth1_cat0_freq_per_this_term,org_lag0_2_auth1_cat0_amount_per_this_term,org_lag0_2_auth1_cat0_instt

### Save Feature

In [78]:
fname = '173_l02'
fname = '174_l02'
ignore_features = ['first_active_month', 'card_id', target]

df_feat.columns = [col.replace('.', '_') for col in df_feat.columns]

for col in df_feat.columns:
    if col in ignore_features: continue
    if (col.count('feature_')):continue
    if (col.count('unix_date')):continue
#     if (col.count('purchase_date')) and not(col.count('diff')):continue
    if (col.count('purchase_date')):continue
    feature = df_feat[col].astype('float32').values
        
#     utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_{col}@', obj=feature)
    utils.to_pkl_gzip(path = f'../features/2_second_valid/{fname}_{col}@', obj=feature)

In [72]:
# なぜかうまくできなかったので個別に

fname = '173_l02'

aggs = {}
aggs['purchase_date'] = ['max', 'min']
max_min_1 = auth1[[key, 'purchase_date']].groupby(key).agg(aggs)
max_min_0 = auth0[[key, 'purchase_date']].groupby(key).agg(aggs)

max_min_1.columns = ['auth1_purchase_date_max', 'auth1_purchase_date_min']
max_min_0.columns = ['auth0_purchase_date_max', 'auth0_purchase_date_min']

max_min = max_min_1.join(max_min_0)
max_min.head()

max_min[f'{term_type}_auth1_0_purchase_date_max_diff'] = (max_min[f'auth1_purchase_date_max'] - max_min[f'auth0_purchase_date_max']).dt.days
max_min[f'{term_type}_auth1_0_purchase_date_min_diff'] = (max_min[f'auth1_purchase_date_min'] - max_min[f'auth0_purchase_date_min']).dt.days

col_list = [ f'{term_type}_auth1_0_purchase_date_max_diff' ,f'{term_type}_auth1_0_purchase_date_min_diff' ]
max_min = train_test.join(max_min)

for col in col_list:
    tmp = max_min[col].fillna(-1).values.astype('float32')
    print(tmp.shape)
    utils.to_pkl_gzip(path = f'../features/2_second_valid/{fname}_{col}@', obj=tmp)

(325540,)
(325540,)


In [81]:
#========================================================================
# City_id
#========================================================================
df = auth1
df['cnt'] = 1
df_city = df.groupby([key, 'city_id'])['cnt'].sum().reset_index()
df_rec = df.groupby([key])['cnt'].sum().reset_index()

main = df_city.merge(df_rec, how='inner', on=key)
del df_city, df_rec
gc.collect()

df['authorized_flag'] = df['authorized_flag'].map(lambda x: 0 if x=='N' else 1)
auth_mean = df.groupby('city_id')['authorized_flag'].mean()

main['main_city_ratio'] = main['cnt_x'] / (main['cnt_y'])
main_city_idx = main.groupby(key)['main_city_ratio'].idxmax()

main_city = main.iloc[main_city_idx][[key, 'city_id', 'main_city_ratio']]
main_city['main_city_auth_mean'] = main_city['city_id'].map(auth_mean)
print(main_city.shape)
main_city.head()

,card_id,city_id,cnt_x,cnt_y
0,C_ID_00007093c1,-1,2,20
1,C_ID_00007093c1,244,18,20
2,C_ID_0001238066,-1,4,36
3,C_ID_0001238066,69,5,36
4,C_ID_0001238066,291,1,36
5,C_ID_0001238066,314,17,36
6,C_ID_0001238066,333,9,36
7,C_ID_0001793786,69,1,19
8,C_ID_0001793786,96,1,19
9,C_ID_0001793786,179,5,19


(234208, 4)


,card_id,city_id,main_city_ratio,main_city_auth_mean
1,C_ID_00007093c1,244,0.900000,1
5,C_ID_0001238066,314,0.472222,1
10,C_ID_0001793786,204,0.631579,1
13,C_ID_000183fdda,161,0.872727,1
17,C_ID_00027503e2,146,0.857143,1


In [82]:
feat_no = '175_l02'

df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
df_train.set_index(key, inplace=True)
df_test.set_index(key, inplace=True)
train_test = pd.concat([df_train, df_test], axis=0)
main_city.set_index(key, inplace=True)

df_feat = train_test.merge(main_city, how='left', on=key)

for col in df_feat.columns:
    if col in ignore_list:continue
    if not(col.count('main_city_ratio')):continue
        
    df_feat[col].fillna(-1, inplace=True)
    feature = df_feat[col].astype('float32').values
    
#     utils.to_pkl_gzip(path = f'../features/1_first_valid/{feat_no}_all_{col}@', obj=feature)
    utils.to_pkl_gzip(path = f'../features/2_second_valid/{feat_no}_{col}@', obj=feature)

100%|██████████| 3/3 [00:00<00:00, 81.18it/s]
